<a href="https://colab.research.google.com/github/Aesol-Ahn/DA_Exercise/blob/master/Chatbot_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install konlpy

     |████████████████████████████████| 19.4MB 240kB/s 
     |████████████████████████████████| 460kB 54.4MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

In [13]:
chatbot_data = pd.read_csv("/content/ChatbotData+.csv")
chatbot_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [16]:
question, answer=list(chatbot_data['Q']), list(chatbot_data['A'])

In [17]:
len(question)

11823

In [18]:
question = question[:100]
answer = answer[:100]

In [19]:
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : PPL 심하네
A : 눈살이 찌푸려지죠.

Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



In [20]:
# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [26]:
def pos_tag(sentences):
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    # 문장 품사 변수 초기화
    sentences_pos = []
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)

        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
#몹스를 이용한 형태소 분석 및 형태소 분석 결과를 " "로 연결, sentences_pos에 저장

    return sentences_pos

#하루가 또 가네요.를 sentence에 넣고 .제거됌
#import re => 정규표현식과 관련된 패키지이다.

In [28]:
#각변수에 저장
question = pos_tag(question)
answer = pos_tag(answer)



In [29]:
question

['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [30]:
answer

['하루 가 또 가네요',
 '위로 해 드립니다',
 '여행 은 언제나 좋죠',
 '여행 은 언제나 좋죠',
 '눈살 이 찌푸려지죠',
 '다시 새로 사는 게 마음 편해요',
 '다시 새로 사는 게 마음 편해요',
 '잘 모르고 있을 수도 있어요',
 '시간 을 정 하고 해보세요',
 '시간 을 정 하고 해보세요',
 '자랑 하는 자리 니까 요',
 '그 사람 도 그럴 거 예요',
 '그 사람 도 그럴 거 예요',
 '혼자 를 즐기세요',
 '돈 은 다시 들어올 거 예요',
 '땀 을 식혀주세요',
 '어서 잊고 새 출발 하세요',
 '빨리 집 에 돌아가서 끄고 나오세요',
 '빨리 집 에 돌아가서 끄고 나오세요',
 '다음 달 에는 더 절약 해봐요',
 '따뜻하게 사세요',
 '다음 달 에는 더 절약 해봐요',
 '가장 확실한 시간 은 오늘이 에요 어제 와 내일 을 놓고 고민 하느라 시간 을 낭비하지 마세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '더 가까워질 기회 가 되겠네요',
 '저 도 요',
 '다 들 바빠서 이야기 할 시간 이 부족했나 봐요',
 '다 들 바빠서 이야기 할 시간 이 부족했나 봐요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '좋은 생각 이에요',
 '더 가까워질 기회 가 되겠네요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '좋은 생각 이에요',
 '정말 후회 할 습관 이에요',
 '무모한 결정 을 내 리지 마세요',
 '선생님 이나 기관 에 연락 해보세요',
 '떨리는 감정 은 그 자체 로 소중해요',
 '득템 했길 바라요

In [31]:
# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 12시 땡
A : 하루 가 또 가네요

Q : 1 지망 학교 떨어졌어
A : 위로 해 드립니다

Q : 3 박 4일 놀러 가고 싶다
A : 여행 은 언제나 좋죠

Q : 3 박 4일 정도 놀러 가고 싶다
A : 여행 은 언제나 좋죠

Q : PPL 심하네
A : 눈살 이 찌푸려지죠

Q : SD 카드 망가졌어
A : 다시 새로 사는 게 마음 편해요

Q : SD 카드 안 돼
A : 다시 새로 사는 게 마음 편해요

Q : SNS 맞팔 왜 안 하지 ㅠㅠ
A : 잘 모르고 있을 수도 있어요

Q : SNS 시간 낭비 인 거 아는데 매일 하는 중
A : 시간 을 정 하고 해보세요

Q : SNS 시간 낭비 인데 자꾸 보게 됨
A : 시간 을 정 하고 해보세요



In [32]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)

In [33]:
sentences #question을 sentences 리스트로 옮김 

['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [34]:
sentences.extend(answer)


In [35]:
sentences #question과 answer가 하나로 저장됌.

['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [36]:
words = []

In [39]:
for sentence in sentences:
  for word in sentence.split(): #공백문자 기준으로 word 저장
    words.append(word)

#길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

#중복된 단어는 삭제
words = list(set(words))


In [40]:
words

['짧죠',
 '나를',
 '가출',
 'SD',
 '갔어',
 '있는',
 '목소리',
 '지망',
 '집',
 '키워',
 '인게',
 '정',
 '오려나',
 '가상',
 '봐서',
 '가지',
 '찌푸려지죠',
 '마세요',
 '수',
 '있어',
 '맛있게',
 '누구',
 '만들어',
 '곧',
 '짠으로',
 '했잖아',
 '엉망',
 '켜고',
 '설움',
 '갑자기',
 '자신',
 '자체',
 '오세요',
 '망함',
 '애',
 'PPL',
 '다음',
 '와',
 '단',
 '좋겠다',
 '돼겠지',
 '가족',
 '도',
 '켜놓고',
 '싶네요',
 '뭘',
 '붙잡고',
 '개인',
 '해도',
 '개강',
 '간다',
 '안',
 '패턴',
 '걸리겠어',
 '뭐',
 '모르고',
 '득템',
 '살찐',
 '먹고',
 '여행',
 '애가',
 '수도',
 '감기',
 '고민',
 '가까워질',
 '줘',
 '먹을까',
 '가보세요',
 '봅니다',
 '좋더라',
 '일도',
 '기회',
 '있으면',
 '떨리는',
 '습관',
 '처음',
 '다',
 '알아차리지',
 '듣고',
 '화폐',
 '살까',
 '혼자',
 '그게',
 '변화',
 '좋겠네요',
 '하겠어',
 '반',
 '사이',
 '진창',
 '끄고',
 '심하네',
 '4일',
 '망가졌어',
 '사세요',
 '옴',
 '함께',
 '시켜',
 '간장',
 '그럴',
 '으로',
 '취미',
 '했길',
 '갈거야',
 '싶은데',
 '업무',
 '봐요',
 '생각',
 '키우고',
 '비싼데',
 '걔',
 '이다',
 '서로',
 '키울',
 '쉬는',
 '갈',
 '모두',
 '강렬한',
 '무시',
 '그건',
 '되겠네요',
 '이라니',
 '좋아',
 '아세요',
 '먹었는데',
 '것',
 '걸린',
 '수영장',
 '놓고',
 '물어보세요',
 '고고',
 '즐거운',
 '벗어나는',
 '추천',
 '개시',
 '서먹해',
 '보게',
 '당황',


In [41]:
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)


In [43]:
# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]


In [44]:
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '짧죠',
 '나를',
 '가출',
 'SD',
 '갔어',
 '있는',
 '목소리',
 '지망',
 '집',
 '키워',
 '인게',
 '정',
 '오려나',
 '가상',
 '봐서',
 '가지',
 '찌푸려지죠',
 '마세요',
 '수',
 '있어',
 '맛있게',
 '누구',
 '만들어',
 '곧',
 '짠으로',
 '했잖아',
 '엉망',
 '켜고',
 '설움',
 '갑자기',
 '자신',
 '자체',
 '오세요',
 '망함',
 '애',
 'PPL',
 '다음',
 '와',
 '단',
 '좋겠다',
 '돼겠지',
 '가족',
 '도',
 '켜놓고',
 '싶네요',
 '뭘',
 '붙잡고',
 '개인',
 '해도',
 '개강',
 '간다',
 '안',
 '패턴',
 '걸리겠어',
 '뭐',
 '모르고',
 '득템',
 '살찐',
 '먹고',
 '여행',
 '애가',
 '수도',
 '감기',
 '고민',
 '가까워질',
 '줘',
 '먹을까',
 '가보세요',
 '봅니다',
 '좋더라',
 '일도',
 '기회',
 '있으면',
 '떨리는',
 '습관',
 '처음',
 '다',
 '알아차리지',
 '듣고',
 '화폐',
 '살까',
 '혼자',
 '그게',
 '변화',
 '좋겠네요',
 '하겠어',
 '반',
 '사이',
 '진창',
 '끄고',
 '심하네',
 '4일',
 '망가졌어',
 '사세요',
 '옴',
 '함께',
 '시켜',
 '간장',
 '그럴',
 '으로',
 '취미',
 '했길',
 '갈거야',
 '싶은데',
 '업무',
 '봐요',
 '생각',
 '키우고',
 '비싼데',
 '걔',
 '이다',
 '서로',
 '키울',
 '쉬는',
 '갈',
 '모두',
 '강렬한',
 '무시',
 '그건',
 '되겠네요',
 '이라니',
 '좋아',
 '아세요',
 '먹었는데',
 '것',
 '걸린',
 '수영장',
 '놓고',
 '물어보세요',
 '고고',
 '즐거운',

In [45]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)} #문장을 인덱스로 변환하여 챗봇 모델에 입력
index_to_word = {index: word for index, word in enumerate(words)} #챗봇 모델의 예측 결과인 인덱스를 문장으로 변환


In [46]:
index_to_word

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '짧죠',
 5: '나를',
 6: '가출',
 7: 'SD',
 8: '갔어',
 9: '있는',
 10: '목소리',
 11: '지망',
 12: '집',
 13: '키워',
 14: '인게',
 15: '정',
 16: '오려나',
 17: '가상',
 18: '봐서',
 19: '가지',
 20: '찌푸려지죠',
 21: '마세요',
 22: '수',
 23: '있어',
 24: '맛있게',
 25: '누구',
 26: '만들어',
 27: '곧',
 28: '짠으로',
 29: '했잖아',
 30: '엉망',
 31: '켜고',
 32: '설움',
 33: '갑자기',
 34: '자신',
 35: '자체',
 36: '오세요',
 37: '망함',
 38: '애',
 39: 'PPL',
 40: '다음',
 41: '와',
 42: '단',
 43: '좋겠다',
 44: '돼겠지',
 45: '가족',
 46: '도',
 47: '켜놓고',
 48: '싶네요',
 49: '뭘',
 50: '붙잡고',
 51: '개인',
 52: '해도',
 53: '개강',
 54: '간다',
 55: '안',
 56: '패턴',
 57: '걸리겠어',
 58: '뭐',
 59: '모르고',
 60: '득템',
 61: '살찐',
 62: '먹고',
 63: '여행',
 64: '애가',
 65: '수도',
 66: '감기',
 67: '고민',
 68: '가까워질',
 69: '줘',
 70: '먹을까',
 71: '가보세요',
 72: '봅니다',
 73: '좋더라',
 74: '일도',
 75: '기회',
 76: '있으면',
 77: '떨리는',
 78: '습관',
 79: '처음',
 80: '다',
 81: '알아차리지',
 82: '듣고',
 83: '화폐',
 84: '살까',
 85: '혼자',
 86: '그게',
 87: '변화',
 88:

In [47]:
question[0]

'12시 땡'

In [52]:
#"12시 땡" => word_to_index => [31, 333] 여기에 패딩까지해줄거야 =>[31, 333, 0, 0,0, ...]
#그렇담 패딩 전체 길이를 어떻게 할 것인가?

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2


In [58]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type):
  sentences_index = [] #문장 -> 단어 -> 인덱스 -> 저장

  # 모든 문장에 대해서 반복
  for sentence in sentences:
    sentence_index = [] #각각의 문장을 구성하는 단어들에 대한 index가 저장되는 리스트 변수
    
    # 디코더 입력일 경우 맨 앞에 START 태그 추가
    if type == DECODER_INPUT:
      sentence_index.extend([vocabulary[STA]])

    # 문장의 단어들을 띄어쓰기로 분리
    for word in sentence.split(): #[12시, 땡]
      if vocabulary.get(word) is not None: #파이썬 문법이 딕셔너리는 키에대한 값이 없으면 none이 나온다.
        # 사전에 있는 단어면 해당 인덱스를 추가
        sentence_index.extend([vocabulary[word]])
      else: # 사전에 없는 단어면 OOV 인덱스를 추가
        sentence_index.extend([vocabulary[OOV]])

    # 최대 길이 검사
    if type == DECODER_TARGET:
      # 디코더 목표일 경우 맨 뒤에 END 태그 추가
      if len(sentence_index) >= max_sequences:
        sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
      else:
        sentence_index += [vocabulary[END]]
    else:
       if len(sentence_index) > max_sequences:
         sentence_index = sentence_index[:max_sequences]
            
    # 최대 길이에 없는 공간은 패딩 인덱스로 채움
    sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]

    sentences_index.append(sentence_index)

  return np.array(sentences_index) #리스트를 np.array로 배열로 바꿈

In [59]:
#인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

#첫 번째 인코더 입력 출력 ("12시 땡")
x_encoder[0]

array([309, 210,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [60]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]


array([  1, 443, 411, 200, 277,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [61]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]


array([443, 411, 200, 277,   2,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [62]:
np.shape(x_encoder)

(100, 30)

In [75]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))


In [76]:
one_hot_data.shape #(100, 30, 454)
y_decoder.shape #(100,30,454)

(100, 30, 454)

In [77]:
for i, sequence in enumerate(y_decoder):
  print(i,sequence)
  
  #각 단어를 454차원 원핫벡터로 표현
  #대괄호가 30쌍 

0 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
1 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
2 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
3 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
4 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
5 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
6 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]


In [79]:
# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
      one_hot_data[i, j, index] = 1   
        #원핫 인코딩
        #i는 100개 문장, j는 30개 단어, index는 454개

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]


IndexError: ignored

In [71]:
#모델링

#훈련 모델 인코더 정의
#훈련 모델 디코더 정의
#예측 모델 인코더 정의
#예측 모델 디코더 정의
#인덱스를 문장으로 변환

In [72]:
# 임베딩 벡터 차원
embedding_dim = 100
# LSTM 히든레이어 차원
lstm_hidden_dim = 128

In [73]:
#훈련 모델 인코더 정의

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



In [80]:
#훈련 모델 디코더 정의

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [81]:
#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [84]:
#--------------------------------------------
# 예측 모델 인코더 정의
#--------------------------------------------

In [83]:
# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

In [85]:
#--------------------------------------------
#예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)


In [86]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence


In [87]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['accuracy'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1
accuracy : 0.8349999785423279
loss : 0.73972487449646


NameError: ignored